In [1]:
def fourier_transform (audio, sampling_rate):
  yf = fft.fft(audio)
  return yf

In [2]:
def feature_divider(feature_vector, feature_name,final_feature_vector, feature_class):
  # check if final_feature_vector empty or not
  if not final_feature_vector:

    for n in range(len(feature_vector[0])):
       final_feature_vector.append({'feature_class': feature_class})

  frame_count = len(final_feature_vector)
  for feature in range(len(feature_vector)):
        # print (feature_vector)
          for frame in range(frame_count):
            name = str(feature_name) + " - " + str(feature) + " - "
            current_frame = final_feature_vector[frame]
            current_frame[name] = feature_vector[feature][frame]

In [3]:
def extract_features(signal, sample_rate, frame_length,hop_length, feature_class,final_feature_vector):
  zeroCrossingRate = librosa.feature.zero_crossing_rate(y=signal, frame_length=frame_length, hop_length=hop_length)
  # print ("zeroCrossingRate", zeroCrossingRate)
  feature_divider(zeroCrossingRate, "zero_crossing_rate",final_feature_vector, feature_class)

  chromaStft = librosa.feature.chroma_stft(y=signal, sr=sample_rate, tuning=0.3, n_fft=frame_length, hop_length=hop_length)
  # print ("chromaStft", chromaStft) # tuning
  feature_divider(chromaStft, "chroma_stft",final_feature_vector, feature_class)

  chromaCqt = librosa.feature.chroma_cqt(y=signal, fmin=10,n_octaves=3, sr=sample_rate, hop_length=hop_length)
  # print ("chromaCqt", chromaCqt) # fmin and n_octaves
  feature_divider(chromaCqt, "chroma_cqt", final_feature_vector, feature_class)

  chromaCens = librosa.feature.chroma_cens(y=signal, fmin=10,n_octaves=3, sr=sample_rate, hop_length=hop_length)
  # print ("chromaCens", chromaCens)
  feature_divider(chromaCens, "chroma_cens",final_feature_vector, feature_class)

  melspectrogram = librosa.feature.melspectrogram(y=signal, sr=sample_rate, n_mels=30, fmin=10, fmax=150, n_fft=frame_length, hop_length=hop_length)
  # print ("melspectrogram", melspectrogram)
  feature_divider(melspectrogram, "melspectrogram",final_feature_vector, feature_class)

  mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=5, n_mels=30, fmin=10, fmax=150, n_fft=frame_length,hop_length=hop_length)
  # print ("mfcc", mfcc)
  feature_divider(mfcc, "mfcc", final_feature_vector,feature_class)

  rmse = librosa.feature.rms(y=signal, frame_length=frame_length, hop_length=hop_length)
  # print ("rmse", rmse)
  feature_divider(rmse, "rmse", final_feature_vector,feature_class)

  spectralCentroid = librosa.feature.spectral_centroid(y=signal, sr=sample_rate, n_fft=frame_length, hop_length=hop_length)
  # print ("spectralCentroid", spectralCentroid)
  feature_divider(spectralCentroid, "spectral_centroid",final_feature_vector, feature_class)

  spectral_bandwidth = librosa.feature.spectral_bandwidth(y=signal, sr=sample_rate,n_fft=frame_length, hop_length=hop_length)
  # print ("spectral_bandwidth", spectral_bandwidth)
  feature_divider(spectral_bandwidth, "spectral_bandwidth",final_feature_vector, feature_class)

  spectral_contrast = librosa.feature.spectral_contrast(y=signal, sr=sample_rate, n_fft=frame_length, hop_length=hop_length, fmin=5,n_bands=5)
  # print ("spectral_contrast", spectral_contrast)
  feature_divider(spectral_contrast, "spectral_contrast",final_feature_vector, feature_class)

  spectral_rolloff = librosa.feature.spectral_rolloff(y=signal,sr=sample_rate, n_fft=frame_length, hop_length=hop_length,roll_percent=0.85)
  # print ("spectral_rolloff", spectral_rolloff) #roll_percent
  feature_divider(spectral_rolloff, "spectral_rolloff",final_feature_vector, feature_class)

  poly_features = librosa.feature.poly_features(y=signal, sr=sample_rate, n_fft=frame_length, hop_length=hop_length,order=1)
  # print ("poly_features", poly_features)
  feature_divider(poly_features, "poly_features",final_feature_vector, feature_class)

In [4]:
import librosa
import pywt
import copy
import numpy as np
import pandas as pd
from scipy import fft
import matplotlib.pyplot as plt

wavelet_name ='db3'
frame_length = 1000 #1s x sample rate
hop_length = 600 #0.6s x sample rate
features1 = [] #Rumbles Array
features0 = [] #Other Array

In [ ]:
# Rumbles
audioFilePath = "/content/drive/MyDrive/Research/Rumbles"
files1 = librosa.util.find_files(audioFilePath, ext=['wav']) 
files1 = np.asarray(files1)
signal_array = []
for y in files1:
    data, sr = librosa.load(y,sr=1000)
    fourier = fourier_transform(data, sr)
    arr = np.array(fourier).astype(np.float32)
    final_feature_vector = []
    extract_features(arr, 1000, frame_length,hop_length, 1, final_feature_vector)
    features1.append(final_feature_vector)

In [ ]:
# Other
audioFilePath = "/content/drive/MyDrive/Research/NatureSounds"
files0 = librosa.util.find_files(audioFilePath, ext=['wav']) 
files0 = np.asarray(files0)
signal_array = []
for z in files0:
    data, sr = librosa.load(z,sr=1000)
    fourier = fourier_transform(data, sr)
    arr = np.array(fourier).astype(np.float32)
    final_feature_vector = []
    extract_features(arr, 1000, frame_length,hop_length, 0, final_feature_vector)
    features0.append(final_feature_vector)

In [ ]:
features_combined = np.concatenate((features1, features0))

from itertools import chain

df = pd.DataFrame(list(chain.from_iterable(features_combined)))
df.to_csv('/content/drive/MyDrive/Research/features.csv', index=False) 
df